<a href="https://colab.research.google.com/github/Basit2612/Fake-NEWS-Classifier-LSTM/blob/main/Fake_News_Classifier_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
import requests

In [ ]:
df = pd.read_csv('/content/train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.shape

(20800, 5)

In [ ]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
X = df.drop('label', axis=1)

In [ ]:
Y= df['label']

In [ ]:
X.shape

(18285, 4)

In [ ]:
Y.shape

(18285,)

In [ ]:
Y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.15.0'

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [ ]:
voc_size = 5000

In [ ]:
message = X.copy()

In [ ]:
message.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [ ]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
message.reset_index(inplace=True)

In [ ]:
message

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


# Data Preprocessing

In [ ]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
  review = re.sub('[^a-zA-Z]', ' ', message['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
one_hot_repr = [one_hot(words, voc_size) for words in corpus]
one_hot_repr

[[999, 1079, 4928, 4265, 865, 2142, 995, 2065, 3402, 656],
 [2401, 4323, 4989, 3042, 3515, 2534, 3497],
 [3378, 4405, 1323, 433],
 [510, 2561, 4040, 3213, 4467, 3513],
 [3766, 3515, 3327, 3954, 684, 919, 3515, 4964, 4500, 300],
 [4922,
  239,
  3117,
  1802,
  3836,
  1907,
  286,
  2742,
  3763,
  4692,
  8,
  2733,
  2418,
  3317,
  3497],
 [2197, 2912, 451, 1879, 646, 4981, 1184, 4028, 2966, 4790, 269],
 [292, 87, 4519, 1726, 3544, 1541, 1907, 4472, 2966, 4790, 269],
 [606, 4315, 311, 1703, 1153, 2596, 481, 388, 1907, 3855],
 [4239, 4416, 1192, 2574, 2097, 1118, 2119, 3739],
 [2219, 1018, 289, 2053, 1315, 3652, 616, 3647, 1825, 2039, 3322],
 [3213, 3652, 865, 2596, 1907, 3544],
 [2426, 1217, 3976, 2366, 4391, 4023, 4055, 1653, 4841],
 [3480, 431, 2920, 2089, 2860, 2637, 2441, 2966, 4790, 269],
 [4034, 1038, 1719, 248, 2901, 2966, 4790, 269],
 [3906, 1575, 363, 4634, 899, 1606, 1798, 2698, 1801, 2853],
 [4156, 3175, 4323],
 [2950, 1367, 1671, 2925, 1907, 4497, 1611, 3497],
 [3199, 40

# Embedding Representation

In [ ]:
sent_length= 20
embedded_docs = pad_sequences(one_hot_repr, padding = 'post', maxlen=sent_length)
print(embedded_docs)

[[ 999 1079 4928 ...    0    0    0]
 [2401 4323 4989 ...    0    0    0]
 [3378 4405 1323 ...    0    0    0]
 ...
 [4309 4067  780 ...    0    0    0]
 [ 367 3544  686 ...    0    0    0]
 [ 966  661 2921 ...    0    0    0]]


In [ ]:
embedded_docs[0]

array([ 999, 1079, 4928, 4265,  865, 2142,  995, 2065, 3402,  656,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

# Creating Model

In [ ]:
embedding_vector_feature = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_feature, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
len(embedded_docs), Y.shape

(18285, (18285,))

In [ ]:
import numpy as np
X_final = np.array(embedded_docs)
Y_final = np.array(Y)

In [ ]:
X_final, Y_final

(array([[ 999, 1079, 4928, ...,    0,    0,    0],
        [2401, 4323, 4989, ...,    0,    0,    0],
        [3378, 4405, 1323, ...,    0,    0,    0],
        ...,
        [4309, 4067,  780, ...,    0,    0,    0],
        [ 367, 3544,  686, ...,    0,    0,    0],
        [ 966,  661, 2921, ...,    0,    0,    0]], dtype=int32),
 array([1, 0, 1, ..., 0, 1, 1]))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, random_state=42, test_size=0.33)

In [ ]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 12s 49ms/step - loss: 0.3125 - accuracy: 0.8461 - val_loss: 0.1996 - val_accuracy: 0.9191
Epoch 2/10
192/192 [==============================] - 11s 60ms/step - loss: 0.1374 - accuracy: 0.9488 - val_loss: 0.2215 - val_accuracy: 0.9158
Epoch 3/10
192/192 [==============================] - 11s 57ms/step - loss: 0.0955 - accuracy: 0.9664 - val_loss: 0.2351 - val_accuracy: 0.9147
Epoch 4/10
192/192 [==============================] - 17s 90ms/step - loss: 0.0659 - accuracy: 0.9787 - val_loss: 0.2685 - val_accuracy: 0.9145
Epoch 5/10
192/192 [==============================] - 11s 57ms/step - loss: 0.0496 - accuracy: 0.9843 - val_loss: 0.2978 - val_accuracy: 0.9120
Epoch 6/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0396 - accuracy: 0.9882 - val_loss: 0.3169 - val_accuracy: 0.9056
Epoch 7/10
192/192 [==============================] - 11s 56ms/step - loss: 0.0321 - accuracy: 0.9909 - val_loss: 0.4232 - val_accuracy: 

In [ ]:
from tensorflow.keras.layers import Dropout
embedding_vector_feature = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_feature, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
y_pred = model.predict(X_test)
y_pred = np.where(y_pred >0.5, 1, 0)

189/189 [==============================] - 3s 11ms/step


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, y_pred)

0.43247721623860813

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3419
           1       0.43      1.00      0.60      2616

    accuracy                           0.43      6035
   macro avg       0.22      0.50      0.30      6035
weighted avg       0.19      0.43      0.26      6035

